In [2]:
# MODEL
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


In [5]:
batch_size = 16
from keras.preprocessing.image import ImageDataGenerator
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r'C:\Users\datta\Desktop\tfidf\train',  # this is the target directory
        target_size=(300, 300),  # all images will be resized to 300x300
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r'C:\Users\datta\Desktop\tfidf\validation',
        target_size=(300, 300),
        batch_size=batch_size,
        class_mode='binary')

Found 1344 images belonging to 2 classes.
Found 483 images belonging to 2 classes.


In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=400 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=160 // batch_size)

model.save_weights('50_epochs.h5')  # always save your weights after training or during training

Epoch 1/50
25/25 [==============================] - 26s 1s/step - loss: 0.4695 - accuracy: 0.8500 - val_loss: 0.0189 - val_accuracy: 0.9875
Epoch 2/50
25/25 [==============================] - 28s 1s/step - loss: 0.0403 - accuracy: 0.9900 - val_loss: 5.7609e-05 - val_accuracy: 0.9812
Epoch 3/50
25/25 [==============================] - 33s 1s/step - loss: 0.0428 - accuracy: 0.9900 - val_loss: 0.0324 - val_accuracy: 1.0000
Epoch 4/50
25/25 [==============================] - 33s 1s/step - loss: 0.1037 - accuracy: 0.9900 - val_loss: 1.4825e-07 - val_accuracy: 0.9864
Epoch 5/50
25/25 [==============================] - 31s 1s/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 2.4127e-07 - val_accuracy: 0.9937
Epoch 6/50
25/25 [==============================] - 32s 1s/step - loss: 0.1844 - accuracy: 0.9850 - val_loss: 0.0241 - val_accuracy: 1.0000
Epoch 7/50
25/25 [==============================] - 31s 1s/step - loss: 0.0197 - accuracy: 0.9900 - val_loss: 5.2899e-04 - val_accuracy: 0.9864
Epoc

KeyboardInterrupt: 